In [ ]:
import numpy as np
import pandas as pd
import os
import json

Some category selected as they are similar to drugs. and new dataset is created df_drugs.csv.
# !!!! the file  is so big it's care about. Read the filtered file if needed.

In [ ]:
file_path = r'arXiv_Data_files\archive\arxiv-metadata-oai-snapshot.json'
papers = []
with open(file_path , 'r') as f :
    for i , line in enumerate(f) : 
        paper = json.loads(line)
        papers.append(paper)

df = pd.DataFrame(papers)
categories = pd.unique(df.categories)
drugs_cats = ["q-bio.BM", "q-bio.MN", "q-bio.GN", "q-bio.QM"]
df_drugs= df.loc[df['categories'].isin(drugs_cats) ]
df_drugs.to_csv('arXiv_Data_files\df_drugs.csv')

In [ ]:
# Category   	Meaning	Related 
###############################
# q-bio.BM	    Biomolecules
# q-bio.GN	    Genomics	
# q-bio.MN	    Molecular Networks	
# q-bio.QM	    Quantitative Methods	

4831

In [ ]:
df_drugs= pd.read_csv('arXiv_Data_files\df_drugs.csv')

In [ ]:
df_drugs.sort_values('update_date' , ascending=False)
df.categories

In [67]:
import re
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer


def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_drugs['clean_abstract'] = df_drugs['abstract'].apply(clean_text)



In [ ]:

le = LabelEncoder()
df_drugs['label'] = le.fit_transform(df_drugs['categories'])  
le.inverse_transform([0])  

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_drugs['clean_abstract'])


X_train, X_test, y_train, y_test = train_test_split(X, df_drugs['label'], test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

    q-bio.BM       0.80      0.88      0.84       232
    q-bio.GN       0.82      0.72      0.77       181
    q-bio.MN       0.84      0.73      0.78       181
    q-bio.QM       0.76      0.81      0.79       373

    accuracy                           0.79       967
   macro avg       0.80      0.78      0.79       967
weighted avg       0.80      0.79      0.79       967



In [69]:
joblib.dump(clf, 'classifier.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(le, 'label_encoder.pkl')

text = "We analyze protein structure using..."
vector = vectorizer.transform([clean_text(text)])
label = clf.predict(vector)
print(le.inverse_transform(label))  


['q-bio.BM']
